In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import io
import numpy as np

from tqdm import tqdm
from itertools import islice


def load_vectors(fname, limit):
    fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in tqdm(islice(fin, limit), total = limit):
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
    return data

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip -O crawl-300d-2M.vec.zip
!unzip crawl-300d-2M.vec.zip

--2022-05-30 11:15:01--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  52.9MB/s    in 27s     

2022-05-30 11:15:28 (54.6 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       


In [ ]:
vecs = load_vectors('crawl-300d-2M.vec', 100000)

100%|██████████| 100000/100000 [00:08<00:00, 12164.21it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/text_nlp/dataset 2 (1).8m.csv')
df

,PdId,IncidntNum,Incident Code,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,...,Fix It Zones as of 2017-11-06 2 2,DELETE - HSOC Zones 2 2,Fix It Zones as of 2018-02-07 2 2,"CBD, BID and GBD Boundaries as of 2017 2 2","Areas of Vulnerability, 2016 2 2",Central Market/Tenderloin Boundary 2 2,Central Market/Tenderloin Boundary Polygon - Updated 2 2,HSOC Zones as of 2018-06-05 2 2,OWED Public Spaces 2 2,Neighborhoods 2
0,3114751606302,31147516,6302,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,09/28/2003,10:00,SOUTHERN,NONE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5069701104134,50697011,4134,ASSAULT,BATTERY,Wednesday,06/22/2005,12:20,NORTHERN,NONE,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,97.0
2,6074729204104,60747292,4104,ASSAULT,ASSAULT,Saturday,07/15/2006,00:55,CENTRAL,NONE,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,106.0
3,7103536315201,71035363,15201,ASSAULT,STALKING,Tuesday,09/25/2007,00:01,TARAVAL,NONE,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,49.0
4,11082415274000,110824152,74000,MISSING PERSON,MISSING ADULT,Saturday,09/24/2011,11:00,TARAVAL,LOCATED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129520,17050730416710,170507304,16710,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Thursday,06/22/2017,01:42,CENTRAL,"ARREST, CITED",...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,77.0
2129521,17071435504014,170714355,4014,ASSAULT,AGGRAVATED ASSAULT WITH BODILY FORCE,Friday,09/01/2017,17:00,PARK,NONE,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,112.0
2129522,17053348465010,170533484,65010,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Friday,06/30/2017,17:50,BAYVIEW,"ARREST, BOOKED",...,NaN,NaN,22.0,NaN,1.0,NaN,NaN,NaN,NaN,56.0
2129523,17102776330130,171027763,30130,OTHER OFFENSES,PEDDLING WITHOUT A LICENSE,Wednesday,12/20/2017,15:41,SOUTHERN,"ARREST, BOOKED",...,3.0,1.0,3.0,7.0,2.0,1.0,1.0,1.0,35.0,21.0


In [ ]:
df.columns

Index(['PdId', 'IncidntNum', 'Incident Code', 'Category', 'Descript',
       'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X',
       'Y', 'location', 'SF Find Neighborhoods 2 2',
       'Current Police Districts 2 2', 'Current Supervisor Districts 2 2',
       'Analysis Neighborhoods 2 2', 'DELETE - Fire Prevention Districts 2 2',
       'DELETE - Police Districts 2 2', 'DELETE - Supervisor Districts 2 2',
       'DELETE - Zip Codes 2 2', 'DELETE - Neighborhoods 2 2',
       'DELETE - 2017 Fix It Zones 2 2',
       'Civic Center Harm Reduction Project Boundary 2 2',
       'Fix It Zones as of 2017-11-06  2 2', 'DELETE - HSOC Zones 2 2',
       'Fix It Zones as of 2018-02-07 2 2',
       'CBD, BID and GBD Boundaries as of 2017 2 2',
       'Areas of Vulnerability, 2016 2 2',
       'Central Market/Tenderloin Boundary 2 2',
       'Central Market/Tenderloin Boundary Polygon - Updated 2 2',
       'HSOC Zones as of 2018-06-05 2 2', 'OWED Public Spaces 2 2',
       

In [ ]:
df = df.loc[:, df.columns[3]:df.columns[10]]

In [ ]:
df = df.iloc[:1000000]

In [ ]:
df

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address
0,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,09/28/2003,10:00,SOUTHERN,NONE,1ST ST / BRANNAN ST
1,ASSAULT,BATTERY,Wednesday,06/22/2005,12:20,NORTHERN,NONE,1200 Block of EDDY ST
2,ASSAULT,ASSAULT,Saturday,07/15/2006,00:55,CENTRAL,NONE,1600 Block of POWELL ST
3,ASSAULT,STALKING,Tuesday,09/25/2007,00:01,TARAVAL,NONE,400 Block of ULLOA ST
4,MISSING PERSON,MISSING ADULT,Saturday,09/24/2011,11:00,TARAVAL,LOCATED,3200 Block of SAN JOSE AV
...,...,...,...,...,...,...,...,...
999995,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,07/08/2003,19:30,NORTHERN,"ARREST, CITED",800 Block of GROVE ST
999996,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Saturday,09/29/2012,10:00,TARAVAL,NONE,100 Block of THRIFT ST
999997,FRAUD,FRAUDULENT USE OF AUTOMATED TELLER CARD,Monday,08/23/2010,18:00,TENDERLOIN,NONE,0 Block of POWELL ST
999998,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,03/24/2003,14:00,RICHMOND,NONE,PINE ST / BRODERICK ST


In [ ]:
for i in range(df.shape[0]):
    a, b, c = df.at[i, 'Date'].split('/')
    df.at[i, 'Date'] = b + '.' + a + '.' + c

In [ ]:
df

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address
0,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,28.09.2003,10:00,SOUTHERN,NONE,1ST ST / BRANNAN ST
1,ASSAULT,BATTERY,Wednesday,22.06.2005,12:20,NORTHERN,NONE,1200 Block of EDDY ST
2,ASSAULT,ASSAULT,Saturday,15.07.2006,00:55,CENTRAL,NONE,1600 Block of POWELL ST
3,ASSAULT,STALKING,Tuesday,25.09.2007,00:01,TARAVAL,NONE,400 Block of ULLOA ST
4,MISSING PERSON,MISSING ADULT,Saturday,24.09.2011,11:00,TARAVAL,LOCATED,3200 Block of SAN JOSE AV
...,...,...,...,...,...,...,...,...
999995,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,08.07.2003,19:30,NORTHERN,"ARREST, CITED",800 Block of GROVE ST
999996,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Saturday,29.09.2012,10:00,TARAVAL,NONE,100 Block of THRIFT ST
999997,FRAUD,FRAUDULENT USE OF AUTOMATED TELLER CARD,Monday,23.08.2010,18:00,TENDERLOIN,NONE,0 Block of POWELL ST
999998,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,24.03.2003,14:00,RICHMOND,NONE,PINE ST / BRODERICK ST


In [ ]:
df['title'] = df['Category']+ ' on ' + df['Date']

In [ ]:
df['body'] = df['Descript'] + ' at ' + df['DayOfWeek'] + ' ' + df['Time'] + ' in ' + df['PdDistrict'] + ' ' + df['Address'] + '.Resolution: ' + df['Resolution']

In [ ]:
df

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,title,body
0,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Sunday,28.09.2003,10:00,SOUTHERN,NONE,1ST ST / BRANNAN ST,LARCENY/THEFT on 28.09.2003,PETTY THEFT FROM A BUILDING at Sunday 10:00 in...
1,ASSAULT,BATTERY,Wednesday,22.06.2005,12:20,NORTHERN,NONE,1200 Block of EDDY ST,ASSAULT on 22.06.2005,BATTERY at Wednesday 12:20 in NORTHERN 1200 Bl...
2,ASSAULT,ASSAULT,Saturday,15.07.2006,00:55,CENTRAL,NONE,1600 Block of POWELL ST,ASSAULT on 15.07.2006,ASSAULT at Saturday 00:55 in CENTRAL 1600 Bloc...
3,ASSAULT,STALKING,Tuesday,25.09.2007,00:01,TARAVAL,NONE,400 Block of ULLOA ST,ASSAULT on 25.09.2007,STALKING at Tuesday 00:01 in TARAVAL 400 Block...
4,MISSING PERSON,MISSING ADULT,Saturday,24.09.2011,11:00,TARAVAL,LOCATED,3200 Block of SAN JOSE AV,MISSING PERSON on 24.09.2011,MISSING ADULT at Saturday 11:00 in TARAVAL 320...
...,...,...,...,...,...,...,...,...,...,...
999995,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,08.07.2003,19:30,NORTHERN,"ARREST, CITED",800 Block of GROVE ST,OTHER OFFENSES on 08.07.2003,TRAFFIC VIOLATION at Tuesday 19:30 in NORTHERN...
999996,LARCENY/THEFT,GRAND THEFT FROM A BUILDING,Saturday,29.09.2012,10:00,TARAVAL,NONE,100 Block of THRIFT ST,LARCENY/THEFT on 29.09.2012,GRAND THEFT FROM A BUILDING at Saturday 10:00 ...
999997,FRAUD,FRAUDULENT USE OF AUTOMATED TELLER CARD,Monday,23.08.2010,18:00,TENDERLOIN,NONE,0 Block of POWELL ST,FRAUD on 23.08.2010,FRAUDULENT USE OF AUTOMATED TELLER CARD at Mon...
999998,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,24.03.2003,14:00,RICHMOND,NONE,PINE ST / BRODERICK ST,VANDALISM on 24.03.2003,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES at M..."


In [ ]:
df = df.drop(['Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address'], axis=1)

In [ ]:
df = df.dropna(how='any')

In [ ]:
df

,title,body
0,LARCENY/THEFT on 28.09.2003,PETTY THEFT FROM A BUILDING at Sunday 10:00 in...
1,ASSAULT on 22.06.2005,BATTERY at Wednesday 12:20 in NORTHERN 1200 Bl...
2,ASSAULT on 15.07.2006,ASSAULT at Saturday 00:55 in CENTRAL 1600 Bloc...
3,ASSAULT on 25.09.2007,STALKING at Tuesday 00:01 in TARAVAL 400 Block...
4,MISSING PERSON on 24.09.2011,MISSING ADULT at Saturday 11:00 in TARAVAL 320...
...,...,...
999995,OTHER OFFENSES on 08.07.2003,TRAFFIC VIOLATION at Tuesday 19:30 in NORTHERN...
999996,LARCENY/THEFT on 29.09.2012,GRAND THEFT FROM A BUILDING at Saturday 10:00 ...
999997,FRAUD on 23.08.2010,FRAUDULENT USE OF AUTOMATED TELLER CARD at Mon...
999998,VANDALISM on 24.03.2003,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES at M..."


In [ ]:
import pickle

In [ ]:
with open('vecs.pickle', 'wb') as f:
  pickle.dump(vecs, f)

In [ ]:
df.to_csv('dataframe.csv')

In [ ]:
from google.colab import files

In [ ]:
files.download('/content/dataframe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/vecs.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>